# Toronto Child Care Centres
---

[Data Source](https://www.toronto.ca/data/children/dmc/a2z/a2za.html)

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
from bs4 import BeautifulSoup as bs

In [6]:
import urllib

In [7]:
import string

In [8]:
import geocoder

In [9]:
page_index = []

In [10]:
for letter in string.ascii_lowercase:
    page_index.append(letter)

In [11]:
page_index.remove('x')
page_index.remove('z')
print(page_index)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [12]:
pages = []

In [13]:
for letter in page_index:
    pages.append('https://www.toronto.ca/data/children/dmc/a2z/a2z' + letter + '.html')

In [14]:
pages

['https://www.toronto.ca/data/children/dmc/a2z/a2za.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zb.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zc.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zd.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2ze.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zf.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zg.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zh.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zi.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zj.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zk.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zl.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zm.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zn.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zo.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zp.html',
 'https://www.toronto.ca/data/children/dmc/a2z/a2zq.html

In [15]:
centres = []

In [16]:
for page in pages:
    link_html = urllib.request.urlopen(urllib.request.Request(page)).read()
    soup_obj = bs(link_html, 'html.parser')
      
    for link in soup_obj.findAll('a'):
        centres.append(link.get('href'))

In [17]:
len(centres)

1870

In [18]:
centresfiltered = [centre for centre in centres if 'webreg' in centre]

In [20]:
len(centresfiltered)

1030

In [21]:
centresfiltered[1000:1004]

['../webreg/gcreg4110.html',
 '../webreg/gcreg1938.html',
 '../webreg/gcreg13586.html',
 '../webreg/gcreg1934.html']

In [22]:
centre_links = []

In [23]:
for i in centresfiltered:
    centre_links.append('https://www.toronto.ca/data/children/dmc/' + i[3:])

In [24]:
centre_links[1000:1004]

['https://www.toronto.ca/data/children/dmc/webreg/gcreg4110.html',
 'https://www.toronto.ca/data/children/dmc/webreg/gcreg1938.html',
 'https://www.toronto.ca/data/children/dmc/webreg/gcreg13586.html',
 'https://www.toronto.ca/data/children/dmc/webreg/gcreg1934.html']

In [73]:
name = []
address = []
coordinates = []
latitude = []
longitude = []

In [74]:
%%time

counter = 0

for i in centre_links:
    
    link_html = urllib.request.urlopen(urllib.request.Request(i)).read()
    soup_obj = bs(link_html, 'html.parser')
    
    nameX = soup_obj.findAll("h2", {"class", "csd_title"})[0].text[:-7]
    name.append(nameX)
      
    address1 = soup_obj.find("p").text
    address2 = address1.replace("\r", "")
    address3 = address2.replace("\t", "")
    address4 = address3.replace("\n", "")
    address5 = address4.replace("\xa0", "")
    address6 = address5.replace(")", "")
    address7 = address6.split("(")
    addressX = address7[0] + ', Toronto, Ontario'
    address.append(addressX)
    
    g = geocoder.google(addressX)
    coordinateX = g.latlng
    coordinates.append(coordinateX)
    
    try:
        latitudeX = coordinateX[0]
    except:
        latitudeX = None
        
    latitude.append(latitudeX)
    
    try:
        longitudeX = coordinateX[1]
    except:
        longitudeX = None
        
    longitude.append(longitudeX)

    counter = counter + 1
    
    print(counter,". " ,nameX,"\t",addressX,"\t",coordinateX,"\t",latitudeX,"\t",longitudeX)


1 .  A & A Child Care Centre 	 3380 Midland Ave,2nd Floor, Toronto, Ontario 	 None 	 None 	 None
2 .  Abacus Day Care Centre (629735 Ontario Inc.) 	 20 Chichester Pl, Toronto, Ontario 	 [43.7773326, -79.321778] 	 43.7773326 	 -79.321778
3 .  Abacus Montessori Learning Centre 	 1 Credit Union Dr, Toronto, Ontario 	 None 	 None 	 None
4 .  ABC Academy - 18 Melrose Avenue  	 18 Melrose Ave, Toronto, Ontario 	 None 	 None 	 None
5 .  ABC Academy 2 - 20 Melrose Avenue  	 20 Melrose Ave, Toronto, Ontario 	 None 	 None 	 None
6 .  ABC Academy Bayview - 1432 Bayview Avenue  	 1432 Bayview Ave, Toronto, Ontario 	 [43.7021396, -79.3739655] 	 43.7021396 	 -79.3739655
7 .  ABC Academy - Beaches  	 1923 Queen St E, Toronto, Ontario 	 [43.6690063, -79.30405999999999] 	 43.6690063 	 -79.30405999999999
8 .  ABC Academy-McRae  	 235 McRae Dr, Toronto, Ontario 	 [43.7088652, -79.3645519] 	 43.7088652 	 -79.3645519
9 .  A B C  Day Nursery 	 145 Marlee Ave, Toronto, Ontario 	 None 	 None 	 None
10 .  Abso

In [75]:
len(name)

1030

In [76]:
len(address)

1030

In [77]:
len(coordinates)

1030

In [78]:
len(latitude)

1030

In [79]:
len(longitude)

1030

In [80]:
['' if x == None else x for x in longitude]

['',
 -79.321778,
 '',
 '',
 '',
 -79.3739655,
 -79.30405999999999,
 -79.3645519,
 '',
 -79.18331309999999,
 '',
 '',
 -79.4310931,
 -79.30839619999999,
 -79.2940155,
 -79.2786325,
 -79.3093207,
 -79.4790013,
 -79.5869848,
 -79.5943471,
 '',
 -79.5511436,
 '',
 -79.3999965,
 -79.33604799999999,
 -79.3383705,
 -79.41291319999999,
 -79.2709039,
 -79.39822579999999,
 -79.4203883,
 -79.1897843,
 -79.4659762,
 -79.466976,
 -79.2656386,
 -79.4792911,
 -79.48687559999999,
 -79.4869966,
 -79.4049102,
 -79.40576639999999,
 -79.5232832,
 -79.2724616,
 -79.4848294,
 -79.2114541,
 -79.29730200000002,
 -79.35550719999999,
 -79.233043,
 -79.419513,
 -79.4784363,
 -79.3221581,
 -79.43638229999999,
 -79.4400665,
 -79.32385239999999,
 -79.4318713,
 -79.2754163,
 -79.4160122,
 -79.5518165,
 -79.4319356,
 -79.2898302,
 -79.4221474,
 '',
 -79.2788527,
 -79.443906,
 -79.2467889,
 -79.3008241,
 -79.39081329999999,
 -79.351044,
 '',
 -79.2878576,
 -79.3212939,
 -79.17276559999999,
 -79.36962299999999,
 '',
 

In [81]:
['' if x == None else x for x in latitude]

['',
 43.7773326,
 '',
 '',
 '',
 43.7021396,
 43.6690063,
 43.7088652,
 '',
 43.7748078,
 '',
 '',
 43.7385597,
 43.7620932,
 43.8129304,
 43.7867906,
 43.8034745,
 43.7309472,
 43.7407263,
 43.731316,
 '',
 43.6033295,
 '',
 43.650126,
 43.668886,
 43.7742722,
 43.7096995,
 43.8085631,
 43.6457306,
 43.7054909,
 43.8089464,
 43.7330372,
 43.732938,
 43.80990269999999,
 43.6566752,
 43.6548174,
 43.6548769,
 43.6732942,
 43.6659727,
 43.7735331,
 43.79831189999999,
 43.6591685,
 43.7453984,
 43.6792427,
 43.80959680000001,
 43.758129,
 43.740984,
 43.7469968,
 43.7700247,
 43.6786644,
 43.6737838,
 43.6711608,
 43.67310490000001,
 43.684152,
 43.7219391,
 43.7226446,
 43.7214357,
 43.6764445,
 43.7339902,
 '',
 43.825423,
 43.726593,
 43.7888379,
 43.7811984,
 43.7815583,
 43.760966,
 '',
 43.6806291,
 43.6859133,
 43.7759472,
 43.693568,
 '',
 '',
 '',
 43.7986425,
 '',
 '',
 '',
 43.7292858,
 '',
 43.7885762,
 43.6747954,
 '',
 '',
 43.8296486,
 '',
 '',
 43.6566752,
 '',
 43.683629

In [105]:
df = pd.DataFrame({'name': name, 'address': address, 'coordinates': coordinates, 'latitude': latitude, 'longitude': longitude}, columns = ['name', 'address', 'coordinates', 'latitude', 'longitude'])

df.dropna(axis=0, how = 'any', inplace = True)

df.reset_index(inplace=True)

df.drop(['index'], axis = 1, inplace = True)

df.head(5)

,name,address,coordinates,latitude,longitude
0,Abacus Day Care Centre (629735 Ontario Inc.),"20 Chichester Pl, Toronto, Ontario","[43.7773326, -79.321778]",43.777333,-79.321778
1,ABC Academy Bayview - 1432 Bayview Avenue,"1432 Bayview Ave, Toronto, Ontario","[43.7021396, -79.3739655]",43.702140,-79.373965
2,ABC Academy - Beaches,"1923 Queen St E, Toronto, Ontario","[43.6690063, -79.30405999999999]",43.669006,-79.304060
3,ABC Academy-McRae,"235 McRae Dr, Toronto, Ontario","[43.7088652, -79.3645519]",43.708865,-79.364552
4,Absorbent Minds (Op1756925 Ontario Inc.),"16 Old Kingston Rd, Toronto, Ontario","[43.7748078, -79.18331309999999]",43.774808,-79.183313


In [88]:
len(df)

794

In [89]:
import folium
from folium.plugins import MarkerCluster

In [94]:
location = np.mean(df['latitude']), np.mean(df['longitude'])

locationlist = df[['latitude', 'longitude']].values.tolist()

labels = df['name']

map = folium.Map(location = location, 
                 zoom_start=12,
                tiles = 'OpenStreetMap')

marker_cluster = MarkerCluster().add_to(map)

for point in range(len(locationlist)):
                  popup = folium.Popup(labels[point], parse_html = True)
                  folium.Marker(locationlist[point], 
                                popup=popup, 
                               icon = folium.Icon(color = 'blue', icon='fas fa-child', prefix='fa')).add_to(marker_cluster)
display(map)